In [1]:
import os
os.chdir("..")
from src.SparkEnv import sparkSess
import pyspark.sql.functions as F
from pyspark.sql import DataFrameStatFunctions as statFunc
import pandas as pd

Successfully imported Spark Context and Spark Configuration


In [2]:
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, FloatType
from pyspark.sql.functions import UserDefinedFunction, udf, struct
from pyspark.sql.functions import col, when
from pyspark.ml.feature import Bucketizer

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [3]:
pre = "hdfs:///input"

In [4]:
!hadoop fs -ls $pre/

Found 7 items
-rw-r--r--   1 hadoop hadoop    6260261 2020-08-18 16:39 hdfs:///input/featureIndex.csv
drwxr-xr-x   - hadoop hadoop          0 2020-08-18 16:38 hdfs:///input/feature_data
-rw-r--r--   1 hadoop hadoop 2530566106 2020-08-18 16:41 hdfs:///input/ff_dsp_non_addressable_model_aff_tbl.csv
drwxr-xr-x   - hadoop hadoop          0 2020-08-18 16:40 hdfs:///input/gdsa_conversion_data
drwxr-xr-x   - hadoop hadoop          0 2020-08-18 16:40 hdfs:///input/gdsa_data
-rw-r--r--   1 hadoop hadoop 1054158304 2020-08-18 16:39 hdfs:///input/md5_match_data.csv
drwxr-xr-x   - hadoop hadoop          0 2020-08-18 16:39 hdfs:///input/user_requests_10mil


In [ ]:
# !s3-dist-cp --src=s3://gdsa-prague/dsp_LAL/data/ff_dsp_non_addressable_model_aff_tbl.csv.gz --dest=hdfs:///input/gdsa_data/ --outputCodec=None

### Downloading Affinity data in order to have continuous variables to play with

In [5]:
!hadoop fs -ls $pre/

Found 7 items
-rw-r--r--   1 hadoop hadoop    6260261 2020-08-18 16:39 hdfs:///input/featureIndex.csv
drwxr-xr-x   - hadoop hadoop          0 2020-08-18 16:38 hdfs:///input/feature_data
-rw-r--r--   1 hadoop hadoop 2530566106 2020-08-18 16:41 hdfs:///input/ff_dsp_non_addressable_model_aff_tbl.csv
drwxr-xr-x   - hadoop hadoop          0 2020-08-18 16:40 hdfs:///input/gdsa_conversion_data
drwxr-xr-x   - hadoop hadoop          0 2020-08-18 16:40 hdfs:///input/gdsa_data
-rw-r--r--   1 hadoop hadoop 1054158304 2020-08-18 16:39 hdfs:///input/md5_match_data.csv
drwxr-xr-x   - hadoop hadoop          0 2020-08-18 16:39 hdfs:///input/user_requests_10mil


In [6]:
na_values = ['-99', '-98', -99, -98, '\\N', '\\\\N', 'Missing', 'missing', 'NA', '?', '.', 'NULL', '', ' ']


affinity = \
      sparkSess.read.csv(f"{pre}/ff_dsp_non_addressable_model_aff_tbl.csv", header=True, nullValue= na_values)

# acxiom.persist()

affinity.count()

3048983

In [12]:
temp= affinity.sample(withReplacement=False, fraction=0.0001, seed=42).select(affinity.columns[:8]).drop('ZETA_ID')


print(temp.count())
temp.printSchema()

temp.show(5)

308
root
 |-- EMAIL_ADDRESS_MD5: string (nullable = true)
 |-- SEG_100001: string (nullable = true)
 |-- SEG_100002: string (nullable = true)
 |-- SEG_100606: string (nullable = true)
 |-- SEG_100433: string (nullable = true)
 |-- SEG_100430: string (nullable = true)
 |-- SEG_100436: string (nullable = true)

+--------------------+----------+----------+----------+----------+----------+----------+
|   EMAIL_ADDRESS_MD5|SEG_100001|SEG_100002|SEG_100606|SEG_100433|SEG_100430|SEG_100436|
+--------------------+----------+----------+----------+----------+----------+----------+
|bfb1fd608bac28492...|       120|         1|         4|       578|       214|       252|
|20f3c2f6be7db4633...|         0|         1|         1|         1|         1|         3|
|3af755c5bf70c4f85...|         2|         0|         2|       116|        64|        23|
|a8d63ac940c56d84c...|         0|         6|         1|         2|         2|         2|
|66c3d68a3b9d452e5...|        87|         2|         8|        52|

### By Default Pyspark puts all the fields as StringType(). I need to cast them as Float. Getting relevant quantiles for each variable and mapping them to a dict

In [23]:
float_columns= []

dict_relevant_percentiles= {}
relevant_percentiles = list(np.arange(0.25, 1, 0.25)) #25th, 50th (median) and 75th

for col in [col for col in temp.columns if col.startswith('SEG')]:
    print('\n', col)
    try:
        
        temp= temp.withColumn(col, temp[col].cast(FloatType()))
        float_columns.append(col)
        
        list_percentiles= temp.approxQuantile(col, relevant_percentiles, 0.01) ### NOTE: last parameter is the relative error: low values imply higher computations
        
        ### Adding the minimum: NOT NEEDED ANYMORE. The bins will be: -Inf, 25th, 50th, 75th, +Inf
        ## Inserting the minimum: I need these for binning the continuous variables
        #list_percentiles.insert(0, temp.agg(F.min(col).alias('min_'+col)).collect()[0]['min_'+col])
        
        dict_relevant_percentiles[col]= list_percentiles
        print('Managed V')
    except:
        
        print('Failed X')
        pass


 SEG_100001
Managed V

 SEG_100002
Managed V

 SEG_100606
Managed V

 SEG_100433
Managed V

 SEG_100430
Managed V

 SEG_100436
Managed V


In [14]:
temp.printSchema()

root
 |-- EMAIL_ADDRESS_MD5: string (nullable = true)
 |-- SEG_100001: float (nullable = true)
 |-- SEG_100002: float (nullable = true)
 |-- SEG_100606: float (nullable = true)
 |-- SEG_100433: float (nullable = true)
 |-- SEG_100430: float (nullable = true)
 |-- SEG_100436: float (nullable = true)



#### Missing Values imputation with Median

In [34]:
# get continuous columns    -->   NOT NEEDED NOW, I ALREADY GOT THE LIST OF FLOATS
# cont_cols = [f.name for f in df.schema.fields if isinstance(f.dataType, FloatType)]
# cont_cols

In [15]:
dict_relevant_percentiles

{'SEG_100001': [3.0, 8.0, 21.0],
 'SEG_100002': [0.0, 1.0, 4.0],
 'SEG_100606': [1.0, 4.0, 11.0],
 'SEG_100433': [37.0, 74.0, 127.0],
 'SEG_100430': [25.0, 50.0, 96.0],
 'SEG_100436': [6.0, 15.0, 37.0]}

In [16]:
idx_median= 1  # it's the index of the median. In this case it s the 3rd number of each list, after min and 25th

for col in float_columns:
    temp= temp.fillna(dict_relevant_percentiles[col][idx_median], subset= [col]) ## taking the middle

### Experiment with one column (1st iteration)

In [49]:
orig_col= 'SEG_100001'
id_col= 'EMAIL_ADDRESS_MD5'
bin_col= 'binned_' + orig_col

bins= dict_relevant_percentiles[orig_col]
bucketizer = Bucketizer(splits= [float('-Inf')]+ bins+ [float('Inf')] ,inputCol=orig_col, outputCol= bin_col)
df_buck = bucketizer.setHandleInvalid("keep").transform(temp.select([id_col , orig_col]))

df_buck= df_buck.withColumn(bin_col, df_buck[bin_col].cast(IntegerType()))

df_buck.show(5)

+--------------------+----------+-----------------+
|   EMAIL_ADDRESS_MD5|SEG_100001|binned_SEG_100001|
+--------------------+----------+-----------------+
|bfb1fd608bac28492...|     120.0|                3|
|20f3c2f6be7db4633...|       0.0|                0|
|3af755c5bf70c4f85...|       2.0|                0|
|a8d63ac940c56d84c...|       0.0|                0|
|66c3d68a3b9d452e5...|      87.0|                3|
+--------------------+----------+-----------------+
only showing top 5 rows



In [50]:
prefix= bin_col

# pivoted = df_buck.groupBy("cont_random_val").pivot("buckets").agg(F.lit(1)).fillna(0)

pivoted = df_buck.groupBy(id_col).pivot(bin_col).agg(F.lit(1).alias(prefix)).fillna(0) #nothing changes
pivoted.show(5)

+--------------------+---+---+---+---+
|   EMAIL_ADDRESS_MD5|  0|  1|  2|  3|
+--------------------+---+---+---+---+
|65c68aca8845efa8c...|  0|  0|  0|  1|
|8bd9486405b376a70...|  1|  0|  0|  0|
|4144c442faf9dfb28...|  1|  0|  0|  0|
|987d0fae534d672fa...|  0|  1|  0|  0|
|878dc1fd37a319eda...|  0|  1|  0|  0|
+--------------------+---+---+---+---+
only showing top 5 rows



In [51]:
prefix= 'SEG_100001'

for col in pivoted.columns[1:]:
    pivoted = pivoted.withColumnRenamed(col, prefix + '_' + col)
    
pivoted.show(5)

+--------------------+------------+------------+------------+------------+
|   EMAIL_ADDRESS_MD5|SEG_100001_0|SEG_100001_1|SEG_100001_2|SEG_100001_3|
+--------------------+------------+------------+------------+------------+
|4144c442faf9dfb28...|           1|           0|           0|           0|
|65c68aca8845efa8c...|           0|           0|           0|           1|
|8bd9486405b376a70...|           1|           0|           0|           0|
|987d0fae534d672fa...|           0|           1|           0|           0|
|878dc1fd37a319eda...|           0|           1|           0|           0|
+--------------------+------------+------------+------------+------------+
only showing top 5 rows



In [52]:
mapping_col_names= dict(zip( list(pivoted.columns[1:]), ( (orig_col + ' ') *4).split(' ')))
mapping_col_names

{'SEG_100001_0': 'SEG_100001',
 'SEG_100001_1': 'SEG_100001',
 'SEG_100001_2': 'SEG_100001',
 'SEG_100001_3': 'SEG_100001'}

In [53]:
dummy_df = temp.drop(orig_col).join(pivoted.select(pivoted.columns[:-1]), on= id_col, how= 'left')

dummy_df.show()

+--------------------+----------+----------+----------+----------+----------+------------+------------+------------+
|   EMAIL_ADDRESS_MD5|SEG_100002|SEG_100606|SEG_100433|SEG_100430|SEG_100436|SEG_100001_0|SEG_100001_1|SEG_100001_2|
+--------------------+----------+----------+----------+----------+----------+------------+------------+------------+
|bfb1fd608bac28492...|       1.0|       4.0|     578.0|     214.0|     252.0|           0|           0|           0|
|20f3c2f6be7db4633...|       1.0|       1.0|       1.0|       1.0|       3.0|           1|           0|           0|
|3af755c5bf70c4f85...|       0.0|       2.0|     116.0|      64.0|      23.0|           1|           0|           0|
|a8d63ac940c56d84c...|       6.0|       1.0|       2.0|       2.0|       2.0|           1|           0|           0|
|66c3d68a3b9d452e5...|       2.0|       8.0|      52.0|      79.0|      14.0|           0|           0|           0|
|62a3e573761d3ea8c...|       1.0|      13.0|     126.0|      66.

#### 2nd iteration, to see if it works in a loop doing the OHE of different columns...

In [54]:
orig_col= 'SEG_100002'
id_col= 'EMAIL_ADDRESS_MD5'
bin_col= 'binned_' + orig_col

bins= dict_relevant_percentiles[orig_col]
bucketizer = Bucketizer(splits= [float('-Inf')]+ bins+ [float('Inf')] ,inputCol=orig_col, outputCol= bin_col)
df_buck = bucketizer.setHandleInvalid("keep").transform(temp.select([id_col , orig_col]))

df_buck= df_buck.withColumn(bin_col, df_buck[bin_col].cast(IntegerType()))

df_buck.show(5)

+--------------------+----------+-----------------+
|   EMAIL_ADDRESS_MD5|SEG_100002|binned_SEG_100002|
+--------------------+----------+-----------------+
|bfb1fd608bac28492...|       1.0|                2|
|20f3c2f6be7db4633...|       1.0|                2|
|3af755c5bf70c4f85...|       0.0|                1|
|a8d63ac940c56d84c...|       6.0|                3|
|66c3d68a3b9d452e5...|       2.0|                2|
+--------------------+----------+-----------------+
only showing top 5 rows



In [55]:
prefix= bin_col

# pivoted = df_buck.groupBy("cont_random_val").pivot("buckets").agg(F.lit(1)).fillna(0)

pivoted = df_buck.groupBy(id_col).pivot(bin_col).agg(F.lit(1).alias(prefix)).fillna(0) #nothing changes
pivoted.show(5)

+--------------------+---+---+---+
|   EMAIL_ADDRESS_MD5|  1|  2|  3|
+--------------------+---+---+---+
|8bd9486405b376a70...|  1|  0|  0|
|65c68aca8845efa8c...|  1|  0|  0|
|4144c442faf9dfb28...|  1|  0|  0|
|987d0fae534d672fa...|  1|  0|  0|
|878dc1fd37a319eda...|  1|  0|  0|
+--------------------+---+---+---+
only showing top 5 rows



In [56]:
prefix= 'SEG_100002'

for col in pivoted.columns[1:]:
    pivoted = pivoted.withColumnRenamed(col, prefix + '_' + col)
    
pivoted.show(5)

+--------------------+------------+------------+------------+
|   EMAIL_ADDRESS_MD5|SEG_100002_1|SEG_100002_2|SEG_100002_3|
+--------------------+------------+------------+------------+
|8bd9486405b376a70...|           1|           0|           0|
|4144c442faf9dfb28...|           1|           0|           0|
|65c68aca8845efa8c...|           1|           0|           0|
|987d0fae534d672fa...|           1|           0|           0|
|878dc1fd37a319eda...|           1|           0|           0|
+--------------------+------------+------------+------------+
only showing top 5 rows



In [58]:
## It works correctly!!

final_dummy = dummy_df.drop(orig_col).join(pivoted.select(pivoted.columns[:-1]), on= id_col, how= 'left')

final_dummy.toPandas().head()

,EMAIL_ADDRESS_MD5,SEG_100606,SEG_100433,SEG_100430,SEG_100436,SEG_100001_0,SEG_100001_1,SEG_100001_2,SEG_100002_1,SEG_100002_2
0,bfb1fd608bac2849224a93caf553ac07,4.0,578.0,214.0,252.0,0,0,0,0,1
1,20f3c2f6be7db4633ed7df028e96b689,1.0,1.0,1.0,3.0,1,0,0,0,1
2,3af755c5bf70c4f85b30c9af929d66cc,2.0,116.0,64.0,23.0,1,0,0,1,0
3,a8d63ac940c56d84c13c25a107f5bffa,1.0,2.0,2.0,2.0,1,0,0,0,0
4,66c3d68a3b9d452e56887cab33d72fb2,8.0,52.0,79.0,14.0,0,0,0,0,1


# Automating the whole process

In [59]:
float_columns= []

dict_relevant_percentiles= {}
relevant_percentiles = list(np.arange(0.25, 1, 0.25)) #25th, 50th (median) and 75th

potential_float_columns= [col for col in temp.columns if col.startswith('SEG')]

def cast_and_get_quantiles(df, potential_float_columns):
    
    temp= df ## naming convention
    for col in potential_float_columns:
        print('\n', col)
        try:

            temp= temp.withColumn(col, temp[col].cast(FloatType()))
            float_columns.append(col)

            list_percentiles= temp.approxQuantile(col, relevant_percentiles, 0.01) ### NOTE: last parameter is the relative error: low values imply higher computations

            ### Adding the minimum: NOT NEEDED ANYMORE. The bins will be: -Inf, 25th, 50th, 75th, +Inf
            ## Inserting the minimum: I need these for binning the continuous variables
            #list_percentiles.insert(0, temp.agg(F.min(col).alias('min_'+col)).collect()[0]['min_'+col])
        
            dict_relevant_percentiles[col]= list_percentiles
            print('Managed V')
        except:

            print('Failed X')
            pass
        
    return (temp, dict_relevant_percentiles, float_columns)

In [60]:
df, dict_relevant_percentiles, float_columns= cast_and_get_quantiles(df= temp, 
                                                                     potential_float_columns= potential_float_columns)


 SEG_100001
Managed V

 SEG_100002
Managed V

 SEG_100606
Managed V

 SEG_100433
Managed V

 SEG_100430
Managed V

 SEG_100436
Managed V


In [79]:
df.show(5)

+--------------------+----------+----------+----------+----------+----------+----------+
|   EMAIL_ADDRESS_MD5|SEG_100001|SEG_100002|SEG_100606|SEG_100433|SEG_100430|SEG_100436|
+--------------------+----------+----------+----------+----------+----------+----------+
|bfb1fd608bac28492...|     120.0|       1.0|       4.0|     578.0|     214.0|     252.0|
|20f3c2f6be7db4633...|       0.0|       1.0|       1.0|       1.0|       1.0|       3.0|
|3af755c5bf70c4f85...|       2.0|       0.0|       2.0|     116.0|      64.0|      23.0|
|a8d63ac940c56d84c...|       0.0|       6.0|       1.0|       2.0|       2.0|       2.0|
|66c3d68a3b9d452e5...|      87.0|       2.0|       8.0|      52.0|      79.0|      14.0|
+--------------------+----------+----------+----------+----------+----------+----------+
only showing top 5 rows



In [81]:
def impute_bucketize_dummy(df, initial_dummy_df, float_columns,   
                         dict_relevant_percentiles,
                         relevant_percentiles = list(np.arange(0.25, 1, 0.25)), 
                         idx_median= 1,
                         id_col= 'EMAIL_ADDRESS_MD5'):


#     idx_median= 1  # it's the index of the median. In this case it s the 2nd number of each list, after  25th
    
    temp= df ## naming convention
    final_dummy_df= initial_dummy_df
    
    final_mapping_names= {}
    
    for col in float_columns:
        
        prefix= orig_col= col
        
        temp= temp.fillna(dict_relevant_percentiles[col][idx_median], subset= [col]) ## taking the middle


        bin_col= 'binned_' + orig_col

        bins= dict_relevant_percentiles[orig_col]
        bucketizer = Bucketizer(splits= [float('-Inf')] + bins+ [float('Inf')] ,inputCol=orig_col, outputCol= bin_col)
        df_buck = bucketizer.setHandleInvalid("keep").transform(temp.select([id_col , orig_col]))

        df_buck= df_buck.withColumn(bin_col, df_buck[bin_col].cast(IntegerType()))
        

        # pivoted = df_buck.groupBy("cont_random_val").pivot("buckets").agg(F.lit(1)).fillna(0)

        pivoted = df_buck.groupBy(id_col).pivot(bin_col).agg(F.lit(1).alias(prefix)).fillna(0) 

        for col in pivoted.columns[1:]:
            pivoted = pivoted.withColumnRenamed(col, prefix + '_' + col)
            
        mapping_col_names= dict(zip( list(pivoted.columns[1:]), ( (orig_col + ' ') *4).split(' ')))

        final_mapping_names.update(mapping_col_names)
        
        final_dummy_df= final_dummy_df.drop(orig_col).join(pivoted.select(pivoted.columns[:-1]), on= id_col, how= 'left')


    return (final_dummy_df, final_mapping_names)

In [82]:
dummy_df, final_mapping_names= impute_bucketize_dummy(df= df, 
                                                      initial_dummy_df=df,
                                                      float_columns= float_columns,   
                                                         dict_relevant_percentiles= dict_relevant_percentiles,
                                                         relevant_percentiles = list(np.arange(0.25, 1, 0.25)), 
                                                         idx_median= 2,
                                                         id_col= 'EMAIL_ADDRESS_MD5')

In [83]:
final_mapping_names

{'SEG_100001_0': 'SEG_100001',
 'SEG_100001_1': 'SEG_100001',
 'SEG_100001_2': 'SEG_100001',
 'SEG_100001_3': 'SEG_100001',
 'SEG_100002_1': 'SEG_100002',
 'SEG_100002_2': 'SEG_100002',
 'SEG_100002_3': 'SEG_100002',
 'SEG_100606_0': 'SEG_100606',
 'SEG_100606_1': 'SEG_100606',
 'SEG_100606_2': 'SEG_100606',
 'SEG_100606_3': 'SEG_100606',
 'SEG_100433_0': 'SEG_100433',
 'SEG_100433_1': 'SEG_100433',
 'SEG_100433_2': 'SEG_100433',
 'SEG_100433_3': 'SEG_100433',
 'SEG_100430_0': 'SEG_100430',
 'SEG_100430_1': 'SEG_100430',
 'SEG_100430_2': 'SEG_100430',
 'SEG_100430_3': 'SEG_100430',
 'SEG_100436_0': 'SEG_100436',
 'SEG_100436_1': 'SEG_100436',
 'SEG_100436_2': 'SEG_100436',
 'SEG_100436_3': 'SEG_100436'}

In [84]:
dummy_df.take(2)

[Row(EMAIL_ADDRESS_MD5='bfb1fd608bac2849224a93caf553ac07', SEG_100001_0=0, SEG_100001_1=0, SEG_100001_2=0, SEG_100002_1=0, SEG_100002_2=1, SEG_100606_0=0, SEG_100606_1=0, SEG_100606_2=1, SEG_100433_0=0, SEG_100433_1=0, SEG_100433_2=0, SEG_100430_0=0, SEG_100430_1=0, SEG_100430_2=0, SEG_100436_0=0, SEG_100436_1=0, SEG_100436_2=0),
 Row(EMAIL_ADDRESS_MD5='20f3c2f6be7db4633ed7df028e96b689', SEG_100001_0=1, SEG_100001_1=0, SEG_100001_2=0, SEG_100002_1=0, SEG_100002_2=1, SEG_100606_0=0, SEG_100606_1=1, SEG_100606_2=0, SEG_100433_0=1, SEG_100433_1=0, SEG_100433_2=0, SEG_100430_0=1, SEG_100430_1=0, SEG_100430_2=0, SEG_100436_0=1, SEG_100436_1=0, SEG_100436_2=0)]

In [86]:
dummy_df.columns

['EMAIL_ADDRESS_MD5',
 'SEG_100001_0',
 'SEG_100001_1',
 'SEG_100001_2',
 'SEG_100002_1',
 'SEG_100002_2',
 'SEG_100606_0',
 'SEG_100606_1',
 'SEG_100606_2',
 'SEG_100433_0',
 'SEG_100433_1',
 'SEG_100433_2',
 'SEG_100430_0',
 'SEG_100430_1',
 'SEG_100430_2',
 'SEG_100436_0',
 'SEG_100436_1',
 'SEG_100436_2']

In [87]:
dummy_df.select(['EMAIL_ADDRESS_MD5',
                 'SEG_100001_0',
                 'SEG_100001_1',
                 'SEG_100001_2',
                 'SEG_100002_1',
                 'SEG_100002_2']).show(3)

+--------------------+------------+------------+------------+------------+------------+
|   EMAIL_ADDRESS_MD5|SEG_100001_0|SEG_100001_1|SEG_100001_2|SEG_100002_1|SEG_100002_2|
+--------------------+------------+------------+------------+------------+------------+
|bfb1fd608bac28492...|           0|           0|           0|           0|           1|
|20f3c2f6be7db4633...|           1|           0|           0|           0|           1|
|3af755c5bf70c4f85...|           1|           0|           0|           1|           0|
+--------------------+------------+------------+------------+------------+------------+
only showing top 3 rows



In [88]:
### Final Display with all the columns

dummy_df.toPandas().head()

,EMAIL_ADDRESS_MD5,SEG_100001_0,SEG_100001_1,SEG_100001_2,SEG_100002_1,SEG_100002_2,SEG_100606_0,SEG_100606_1,SEG_100606_2,SEG_100433_0,SEG_100433_1,SEG_100433_2,SEG_100430_0,SEG_100430_1,SEG_100430_2,SEG_100436_0,SEG_100436_1,SEG_100436_2
0,bfb1fd608bac2849224a93caf553ac07,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
1,20f3c2f6be7db4633ed7df028e96b689,1,0,0,0,1,0,1,0,1,0,0,1,0,0,1,0,0
2,3af755c5bf70c4f85b30c9af929d66cc,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1
3,a8d63ac940c56d84c13c25a107f5bffa,1,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0
4,66c3d68a3b9d452e56887cab33d72fb2,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,1,0
